# Modélisation d'une membrane couplée à une cavitée
> Auteur: Antoine CAILLON

On suppose (voir l'article de *MARC*) que les fréquences des modes de la membrane $f$ sont fonction de $\rho, c, V, T, a$, de telle sorte que $$J_0(w) = \frac{-\chi}{w^2}J_2(w),$$ avec $$\chi=\frac{\pi \rho c^2 a^4}{VT}, ~~ w = \frac{2\pi f a}{c}.$$

Trouver les bonne fréquences en fonction de $\rho, c, V, T, a$ revient à faire une recherche de zéros sur l'équation 1, ce qui n'est pas compatible avec une idée de faire du contrôle actif en temps réel. On va donc précalculer en fonction de $\chi$ les valeurs des racines de l'équation 1, et faire $n$ approximation polynomiale $\mathcal P$ d'ordre 4 sur la $n^{\text{ème}}$ racine afin de pouvoir accéder plus rapidement à la valeur des fréquences.

L'intérêt de l'équation 1 est qu'elle donne la valeur des fréquences des modes de la membrane en fonction du volume, qu'on peut écrire $$V = V_0 - \Delta V,$$ où $\Delta V$ est la variation de volume apportée par le déplacement de la membrane du HP.

En reliant la tension $u$ au déplacement $x$ de la membrane du HP, on peut donc obtenir $$\Delta V = (u \ast h)  A_{HP},$$ où $h$ est une fonction de blanchissement de la fonction de transfert du HP et $A_{HP}$ l'aire de la membrane du HP.

Cela permet de faire le lien entre tension appliquée au HP et fréquence des modes de la membrane du tom:

$$
f_n = \frac{c}{2\pi a}\mathcal P_n \left(\frac{\pi \rho c^2 a^4}{(V_0 - (u \ast h)  A_{HP})T}\right)
$$

In [1]:
from scipy.special import jv
from scipy.optimize import newton
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown

In [2]:
import numpy as np
from scipy.optimize import newton
from scipy.special import jv

def find_roots(fun, n):
    x = 1e-16
    xin = 0
    dx = 1e-3
    
    roots = []
    
    for i in range(n):
        while 1:
            if (abs(fun(x)) > .05) and not xin:
                x += dx
            elif (abs(fun(x)) <= .05) and not xin:
                xin = x
                x += dx
            elif (abs(fun(x)) > .05) and xin:
                xout = x
                roots.append(newton(fun, xin))
                xin = 0
                break
            else:
                x += dx
    return np.array(roots)

def get_xi(rho, c, a, V, T):
    xi = np.pi*rho*c**2*a**4/(V*T)
    return xi

def get_f(xi):
    return lambda w: jv(0,w) + xi/w**2 * jv(2,w)

In [3]:
n_f = 6
N   = 100

xi = np.linspace(0, 20, N)

roots = np.zeros([n_f, N])

for i,x in enumerate(xi):
    f = get_f(x)
    roots[:, i] = find_roots(f, n_f)
    

In [4]:
poly = []

ordre  = 4

for i in range(n_f):
    poly.append(np.polyfit(xi,roots[i,:], ordre))

In [5]:
poly = np.array(poly)

In [7]:
def generate_cxx_code(poly):
    code = "#include \"fillPolyTable.h\"\n\n"
    code += "//Usage:\n"
    code += "//float poly[%d];\n" % (np.product(poly.shape))
    code += "//fillPolyTable(&poly);\n"
    code += "\n//Coef_j of mode_i is poly[%d * mode_i + coef_j]\n" % (ordre+1) 
    code += "//Order %d polynomial approximation of w0, for xi going from %f to %f.\n" % (ordre, np.min(xi), np.max(xi))
    code += "\nvoid fillPolyTable(float* poly)\n{\n"
    for i in range(np.product(poly.shape)):
        code += "\tpoly[%d] = %f;\n" % (i, poly[i//(ordre+1),i%(ordre+1)])
    code += "}"

    with open("fillPolyTable.cpp","w") as output:
        output.write(code)
    with open("fillPolyTable.h","w") as output:
        output.write("#pragma once\n\nvoid fillPolyTable(float* poly);")
        
    return code
    
code = generate_cxx_code(poly)

Markdown("```C++\n"+code+"```")

```C++
#include "fillPolyTable.h"

//Usage:
//float poly[30];
//fillPolyTable(&poly);

//Coef_j of mode_i is poly[5 * mode_i + coef_j]
//Order 4 polynomial approximation of w0, for xi going from 0.000000 to 20.000000.

void fillPolyTable(float* poly)
{
	poly[0] = -0.000002;
	poly[1] = 0.000108;
	poly[2] = -0.004266;
	poly[3] = 0.141605;
	poly[4] = 2.406022;
	poly[5] = -0.000000;
	poly[6] = 0.000010;
	poly[7] = 0.000265;
	poly[8] = 0.011996;
	poly[9] = 5.520011;
	poly[10] = 0.000000;
	poly[11] = 0.000000;
	poly[12] = 0.000038;
	poly[13] = 0.003085;
	poly[14] = 8.653728;
	poly[15] = 0.000000;
	poly[16] = 0.000000;
	poly[17] = 0.000008;
	poly[18] = 0.001220;
	poly[19] = 11.791534;
	poly[20] = 0.000000;
	poly[21] = 0.000000;
	poly[22] = 0.000003;
	poly[23] = 0.000601;
	poly[24] = 14.930918;
	poly[25] = 0.000000;
	poly[26] = 0.000000;
	poly[27] = 0.000001;
	poly[28] = 0.000339;
	poly[29] = 18.071064;
}```